In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('homework') \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


25/03/05 22:15:49 WARN Utils: Your hostname, CairoPC resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/05 22:15:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/05 22:15:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.version

'3.3.2'

In [1]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet -O yellow_tripdata_2024-10.parquet

--2025-03-05 22:14:57--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 65.8.245.171, 65.8.245.178, 65.8.245.50, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.171|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  11.3MB/s    in 6.1s    

2025-03-05 22:15:04 (10.0 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [5]:
hw_patch = "/home/de-zoomcamp-2025/5_batch/code/yellow_tripdata_2024-10.parquet"
df = spark.read.parquet(hw_patch)

In [6]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [7]:
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-09-30 21:30:44|  2024-09-30 21:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [8]:
df = df \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [9]:
df.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [10]:
from pyspark.sql import functions as F
from pyspark.sql import types

In [11]:
# Change data types of columns
df_hw = df.withColumn("VendorID", F.col("VendorID").cast(types.IntegerType())) \
       .withColumn("RatecodeID", F.col("RatecodeID").cast(types.IntegerType())) \
       .withColumn("PULocationID", F.col("PULocationID").cast(types.IntegerType())) \
       .withColumn("DOLocationID", F.col("DOLocationID").cast(types.IntegerType())) \
       .withColumn("passenger_count", F.col("passenger_count").cast(types.IntegerType())) \
       .withColumn("payment_type", F.col("payment_type").cast(types.IntegerType())) \

# Show the updated schema
df_hw.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [12]:
df_hw \
    .repartition(4) \
    .write.parquet('/home/de-zoomcamp-2025/5_batch/data/raw/hw-5/', mode='overwrite')

In [14]:
!ls -lh /home/de-zoomcamp-2025/5_batch/data/raw/hw-5/

total 97M
-rw-r--r-- 1 root root   0 Mar  5 22:20 _SUCCESS
-rw-r--r-- 1 root root 25M Mar  5 22:20 part-00000-15e5dbed-354e-443d-bfdb-c8fad59aedfb-c000.snappy.parquet
-rw-r--r-- 1 root root 25M Mar  5 22:20 part-00001-15e5dbed-354e-443d-bfdb-c8fad59aedfb-c000.snappy.parquet
-rw-r--r-- 1 root root 25M Mar  5 22:20 part-00002-15e5dbed-354e-443d-bfdb-c8fad59aedfb-c000.snappy.parquet
-rw-r--r-- 1 root root 25M Mar  5 22:20 part-00003-15e5dbed-354e-443d-bfdb-c8fad59aedfb-c000.snappy.parquet


In [15]:
df_hw.createOrReplaceTempView('trips_data')

In [34]:
# How many taxi trips were there on the 15th of October?
# Consider only trips that started on the 15th of October.
spark.sql("""
SELECT
    count(1) as trip_count
FROM
    trips_data
WHERE 
    pickup_datetime >= '2024-10-15' and pickup_datetime < '2024-10-16'
""").show()

+----------+
|trip_count|
+----------+
|    128811|
+----------+



In [33]:
# What is the length of the longest trip in the dataset in hours?
spark.sql("""
SELECT
    max(trip_duration) as max_trip_duration
FROM
    (
    SELECT
        (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime)) / 3600 as trip_duration
    FROM
        trips_data
    )
""").show()

+------------------+
| max_trip_duration|
+------------------+
|162.61777777777777|
+------------------+



In [35]:
zones = spark.read.parquet("/home/de-zoomcamp-2025/5_batch/data/zones/*.parquet")

In [36]:
zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [37]:
zones.createOrReplaceTempView('zones')

In [41]:
zones.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [40]:
# Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?
spark.sql("""
SELECT
    Zone as zone_name,
    count(1) as trip_count
FROM
    (
    SELECT
        PULocationID
    FROM
        trips_data
    )
JOIN
    zones
ON
    PULocationID = cast(LocationID as int)
GROUP BY
    Zone
ORDER BY
    trip_count
LIMIT 5
""").show()

+--------------------+----------+
|           zone_name|trip_count|
+--------------------+----------+
|Governor's Island...|         1|
|       Rikers Island|         2|
|       Arden Heights|         2|
|         Jamaica Bay|         3|
| Green-Wood Cemetery|         3|
+--------------------+----------+

